# Building RAG with Qwen2.5

In [ ]:
import json
import torch
import os
import shutil
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA

# Clear any unused GPU memory
torch.cuda.empty_cache()

# Step 0: Clear the Chroma database if it exists
persist_dir = "./chroma.db"
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

# Step 1: Folder for JSON Files
input_folder = "Vietnam-Law-rag_json"

# Step 2: Load All JSON Files and Convert to LangChain Documents
documents = []
for file_name in os.listdir(input_folder):
    if file_name.endswith(".json"):
        file_path = os.path.join(input_folder, file_name)
        base_file_name = os.path.splitext(file_name)[0]  # Remove the extension for `file_id`
        
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        # Convert JSON data to LangChain Document objects
        documents.extend([
            Document(
                page_content=entry["text"],
                metadata={
                    "id": entry["id"],
                    "article": entry["article"],
                    "clause": entry["clause"],
                    "title": entry["title"],
                    "file_id": base_file_name  # Add file name as an identifier
                }
            )
            for entry in data
        ])

print(f"Loaded {len(documents)} documents from {input_folder}.")

# Step 3: Initialize HuggingFace Embeddings
embeddings_model = HuggingFaceEmbeddings()

# Step 4: Create Chroma Vector Store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=persist_dir
)

# Step 5: Load Qwen Model
model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Step 6: Set Device for GPU/CPU
device = 0 if torch.cuda.is_available() else -1

# Step 7: Create a Text-Generation Pipeline with GPU/CPU and Handle Tokenization Warning
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    device=device,
    clean_up_tokenization_spaces=True
)

# Wrap the pipeline for LangChain
hf = HuggingFacePipeline(pipeline=pipe)

# Step 8: Create a Retrieval-Based QA System
qa_chain = RetrievalQA.from_chain_type(
    llm=hf,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# Clean up GPU memory and move model to CPU after training/inference
torch.cuda.empty_cache()
model.to("cpu")  # Move model back to CPU to free up GPU memory
torch.cuda.empty_cache()

print("System is ready for retrieval-based QA!")

# Test query

In [ ]:
# Step 9: Test the RAG System
torch.cuda.empty_cache()
model.to("cuda" if torch.cuda.is_available() else "cpu")

query = "Thuốc giả là gì?"
result = qa_chain({"query": query})

# Print the Result
print("Answer:", result["result"])
print("Source Documents:")
for doc in result["source_documents"]:
    print(f"Metadata: {doc.metadata}")
    print(f"Content: {doc.page_content}\n")

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()